# IMPORTACIÓN DE LIBRERÍAS

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import ctypes

# CREACIÓN DE LA DATA

In [ ]:
############## VARIABLES #################

nombre="elsaiz" # Nombre o clase donde se guardará el archivo (15 clases en total)
carpeta="entrenamiento" # Carpeta donde se guardará el archivo, se empieza con "entrenamiento"
a=0 # Contador para el número de fotos
b=0 # Contador para dar por finalizado la toma de imágenes
c=0 # Contador para obtener 2 tipos de mensajes, VUELVA PRONTO o FOTOS COMPLETAS

############ COORDENADAS DEL RECTÁNGULO ############

punto1a=10
punto1b=470
punto2a=80
punto2b=600

########## COORDENADAS DE LOS CÍRCULOS GUÍAS #######

puntomedio=226+punto1a
puntoiz=puntomedio-33
puntode=puntomedio+33

############ COLORES PARA VISUALIZACIÓN ###########

color_contorno_mano = (0,255,0) # Color verde
color_rectangulo = (0,255,255)  # Color amarillo
color_rojo = (0,0,255)  # Color rojo
color_blanco = (255,255,255)  # Color blanco
color_azul = (255,0,0)  # Color azul

############# UBICACIONES #################

ubicacion3 = (20,50)
tamañoLetra = 1.1 # Tamaño de letra para el número de imágenes
grosorLetra = 2 # Grosor de letra para el número de imágenes

############ INICIO DE LA CAPTURA DE IMÁGENES ############

camera = cv2.VideoCapture(1) # Captura de video desde la webcam
mensaje=ctypes.windll.user32.MessageBoxW(0, "¿COMENZAMOS?", "INFO", 1) # Muestra un cuadro de diálogo
if mensaje==1: # Si acepta el usuario entra a la condicional
    print("TOMA DE FOTOS COMENZANDO".format(a,nombre="angel"))    
    while(True):
        _, frame = camera.read() # Guarda cada frame en la variable "frame"
        frame_rotate  = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE) # Rota la imagen inicial
        frame_blur_rotate = cv2.GaussianBlur(frame_rotate ,(11,11),0) # Realiza el suavizado gaussiano  
        gray  = cv2.cvtColor(frame_blur_rotate, cv2.COLOR_RGB2GRAY)  # Cambia del espacio RGB a espacio gris
        rgb_imagen = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB) # Cambia del espacio gris a espacio RGB
        _,th = cv2.threshold(gray,75,255,cv2.THRESH_BINARY) # Realiza la umbralización simple de la imagen gris
        contornos,jerarquia = cv2.findContours(th,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Halla los contornos de la imagen anterior
        cnts = sorted(contornos,key=cv2.contourArea,reverse=True)[:1]        
        final=cv2.bitwise_and(rgb_imagen,rgb_imagen,mask=th) # Realiza la resta entre la imagen RGB y la máscara
        final_gray= cv2.cvtColor(final,cv2.COLOR_RGB2GRAY) # Cambia del espacio RGB a espacio gris
        clahe=cv2.createCLAHE(clipLimit=3,tileGridSize=(8,8)) # Crea el algoritmo CLAHE
        clahe_img=clahe.apply(final_gray) # Realiza el algoritmo CLAHE
        RGB  = cv2.cvtColor(clahe_img, cv2.COLOR_GRAY2RGB) # Cambia del espacio gris a espacio RGB
        RGB2  = cv2.cvtColor(clahe_img, cv2.COLOR_GRAY2RGB) # Lo mismo de la anterior línea pero para guardar la imagen
        cv2.rectangle(RGB,(punto1b,punto2b),(punto1a,punto2a),color_rectangulo,1) # Dibuja el rectángulo
        cv2.drawContours(RGB, cnts,0, color_contorno_mano) # Dibuja el contorno de la mano
        cv2.circle(RGB,(puntoiz, 368),15,color_rojo,2) # Dibuja los 3 círculos guías
        cv2.circle(RGB,(puntode, 368),15,color_blanco,2)
        cv2.circle(RGB,(puntomedio, 368),15,color_azul,2)
        for cnt in cnts:
            # Encontrar el centro del contorno
            M = cv2.moments(cnt)
            if M["m00"] == 0: 
                M["m00"]=1
            x = int(M["m10"]/M["m00"])
            y = int(M["m01"]/M["m00"])
            cv2.circle(RGB,tuple([x,y]),4,(0,255,0),-1) # Dibuja el centro del contorno de la mano  
        m=cv2.putText(RGB, str(a)+"/200", ubicacion3, cv2.FONT_HERSHEY_TRIPLEX, tamañoLetra, color_rojo, grosorLetra)
        cv2.imshow('Frame Final',RGB) # Muestra la GUI     
        res = cv2.waitKey(1) # Guarda la variable al hacer click de cualquier número
        if res & 0xFF == ord("2"): # Si hace click en el número "2"
            b=2
            c=1
            break
        elif res & 0xFF == ord("1"): # Si hace click en el número "1"
            a=a+1
            imagen_res1 = cv2.resize(RGB2, (227, 227)) # Redimensiona la imagen
            if a==141:
                carpeta="validacion"
            elif a==181:
                carpeta="pruebas"
            cv2.imwrite("C:/Users/Angel Terrones/Desktop/TESIS/1. imagensin/{carpeta}/{nombre}/{}.png".format(a,carpeta=carpeta,nombre=nombre),\
                          imagen_res1) # Guarda la imagen a la siguiente dirección de carpeta             
            print("foto #{} de {nombre} guardada".format(a,nombre=nombre))
            if a==200:
                b=2
                c=2
                break
if b==2:
    camera.release()
    cv2.destroyAllWindows()
    if c==1:
        ctypes.windll.user32.MessageBoxW(0, "VUELVA PRONTO", "GRACIAS", 1)
    elif c==2:
        ctypes.windll.user32.MessageBoxW(0, "FOTOS COMPLETAS", "GRACIAS", 1)